In [ ]:
import numpy as np
from cv2.aruco import Dictionary_create, DetectorParameters, CORNER_REFINE_SUBPIX, detectMarkers
from cv2 import aruco, cvtColor, COLOR_BGR2GRAY, LUT
# import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
from funktionen import *
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import inv as sparse_inv
from scipy.optimize import least_squares
from scipy import stats

In [ ]:
dateien = []
# [dateien.append(i) for i in glob("../../bilderserien/wuerfel/*.jpg")]
[dateien.append(i) for i in glob("../../bilderserien/wuerfel7/*.jpg")]
[dateien.append(i) for i in glob("../../bilderserien/wuerfel8/*.jpg")]
pictures = pd.DataFrame([[i, int(pfad[-6:-4])-1, pfad, None, None, None, None, None, None] for i, pfad in enumerate(dateien)], columns=["img", "kamera",
                        "pfad", "t1", "t2", "t3", "r1", "r2", "r3"]).astype({"kamera": int, "pfad": str, "img": int, "t1": float, "t2": float, "t3": float, "r1": float, "r2": float, "r3": float}).set_index(["img"])
# np.c_[t.T,rod.T]

In [ ]:
pictures

In [ ]:

aruco_dict = Dictionary_create(32, 3)
parameter = DetectorParameters.create()
parameter.cornerRefinementMethod = CORNER_REFINE_SUBPIX
LUT_IN = [0, 158, 216, 255]
LUT_OUT = [0, 22, 80, 176]
lut = np.interp(np.arange(0, 256),
                LUT_IN, LUT_OUT).astype(np.uint8)

In [ ]:
cornersList = []
for j, img in pictures.iterrows():
    imgCV = cv2.imread(img['pfad'])
    gray = cvtColor(imgCV, COLOR_BGR2GRAY)
    tmp_corners, tmp_ids, t = detectMarkers(
        gray, aruco_dict, parameters=parameter)
    for c, i in zip(tmp_corners, tmp_ids):
        size = np.linalg.norm(c[0][0] - c[0][1])
        # print(size)
        if size < 100:
            continue
        for k in range(len(c[0])):
            cornersList.append([j, i[0], k, c[0][k][0], c[0][k][1]])
corners = pd.DataFrame(cornersList, columns=[
                       "img", "marker", "ecke", "x", "y"])
corners.set_index(["img", "marker", "ecke"], inplace=True)

In [ ]:
corners.reset_index()["marker"].unique()

In [ ]:
ca = corners.reset_index().set_index(["marker", "ecke"])
matched = ca.join(ca, lsuffix="_a", rsuffix="_b").query(
    'img_a < img_b')
matched.reset_index(inplace=True)
matched

In [ ]:
coords_wuerfel = pd.read_csv(
    "../../arucoWuerfel.csv").astype({'x': float, 'y': float, 'z': float})
coords_wuerfel.set_index(["marker", "ecke"], inplace=True)
coords = pd.read_csv(
    "./wuerfel_coords.csv").astype({'x': float, 'y': float, 'z': float})
coords.set_index(["marker", "ecke"], inplace=True)
# coords = coords_wuerfel.copy()
coords

In [ ]:
kameras = pd.read_csv("./wuerfel_kameras.csv")
kameras.set_index(["kamera"], inplace=True)
kameras

In [ ]:
def get_kamera(kameras, kid):
    kamera = kameras.iloc[kid]
    mtx = np.array([[kamera["fx"], 0, kamera["cx"]],
                    [0, kamera["fy"], kamera["cy"]],
                    [0, 0, 1]])
    dist = np.array(kamera[["d1", "d2", "d3", "d4", "d5"]])
    return mtx, dist

In [ ]:
def neues_bild(coords, pictures, kameras, corners):
    lp = corners.join(coords, lsuffix="_img", rsuffix="").dropna().join(
        pictures, lsuffix="", rsuffix="_cam")
    lp = lp[lp["r1"].isnull()]
    bild = lp.groupby("img").count().sort_values(
        "x_img", ascending=False).first_valid_index()
    passpunkte = lp.query('img == ' + str(bild)).reset_index()
    pp = passpunkte[['x_img', 'y_img', 'x', 'y', 'z']].values
    if bild is None:
        print("nichts gefunden")
        return pictures
    kid = pictures.query('img == ' + str(bild))['kamera'].values[0]
    k, dist = get_kamera(kameras, kid)

    if len(pp) >= 4:
        _, r, t, _ = cv2.solvePnPRansac(pp[:, 2:], pp[:, :2], k, dist)
        rod, _ = cv2.Rodrigues(r)
        p = pictures.query('img == ' + str(bild))
        p[["t1", "t2", "t3", "r1", "r2", "r3"]] = [
            t[0][0], t[1][0], t[2][0], r[0][0], r[1][0], r[2][0]]
        pictures.update(p)
    return pictures

In [ ]:
for i in range(80):
    pictures = neues_bild(coords, pictures, kameras, corners)

In [ ]:
pictures  # .sort_index()

In [ ]:
def wolke_zeigen(coords, pictures):

    # pip install plotly

    import plotly.graph_objs as go

    coords_img = []
    for _, p in pictures.dropna().iterrows():
        rt = p[['r1', 'r2', 'r3']].to_numpy(dtype=np.float64)
        R = cv2.Rodrigues(rt)[0]
        R = np.linalg.inv(R)
        t = p[['t1', 't2', 't3']].to_numpy(dtype=np.float64).T
        t = -R@t
        coords_img.append(t)
    coords_img = np.array(coords_img)

    # Create a 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(x=coords['x'], y=coords['y'], z=coords['z'], mode='markers', name='Original Points'),
                          go.Scatter3d(x=coords_img[:, 0], y=coords_img[:, 1], z=coords_img[:, 2],
                                       mode='markers', name='Kameras')])

    # Add labels to the plot
    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
                      title='Rotation and Translation Vectors in 3D')
    fig.layout.scene.camera.projection.type = "orthographic"
    fig.show()


wolke_zeigen(coords, pictures)

In [ ]:
def neue_koordinaten(coords, pictures, kameras):
    pic_a = pictures.reset_index().rename(
        columns={"img": "img_a"}).set_index("img_a")
    pic_b = pictures.reset_index().rename(
        columns={"img": "img_b"}).set_index("img_b")
    matched = ca.add_suffix('_a').join(ca.add_suffix('_b')).query(
        'img_a < img_b').join(coords.add_suffix("_ca"))
    # print(matched)
    matched = matched[matched["x_ca"].isnull()].reset_index().set_index([
        "img_a", "img_b"])
    matched = matched.join(pic_a.add_suffix("_a"))
    matched = matched.join(pic_b.add_suffix("_b"))
    # print(matched)
    matched.dropna(subset=['t1_a', 't1_b'], inplace=True)
    matched = matched.reset_index().set_index(["marker", "ecke"])

    beste = matched.groupby(["marker"]).count().sort_values(
        "img_a", ascending=False)
    beste

    # print(beste)

    for bester in beste.reset_index().values[:, 0]:
        new = []
        # print(bester)
        for i in range(4):
            cneu = []

            for i, v in matched.query("marker =="+str(bester)).query("ecke =="+str(i)).iterrows():
                # print(v)
                K1, dist1 = get_kamera(kameras, v['kamera_a'])
                K2, dist2 = get_kamera(kameras, v['kamera_b'])
                R1 = cv2.Rodrigues(
                    np.array(list(v[['r1_a', 'r2_a', 'r3_a']].to_numpy(dtype=np.float64))))[0]
                R2 = cv2.Rodrigues(
                    np.array(list(v[['r1_b', 'r2_b', 'r3_b']].to_numpy(dtype=np.float64))))[0]
                t1 = v[['t1_a', 't2_a', 't3_a']].to_numpy(dtype=np.float64)
                t2 = v[['t1_b', 't2_b', 't3_b']].to_numpy(dtype=np.float64)
                P1 = np.c_[R1, t1]
                P2 = np.c_[R2, t2]
                pt1 = np.array(list(v[['x_a', 'y_a']]))
                pt2 = np.array(list(v[['x_b', 'y_b']]))
                p1n = cart2hom(cv2.undistortPoints(pt1, K1, dist1)[0][0])
                p2n = cart2hom(cv2.undistortPoints(pt2, K2, dist2)[0][0])

                xy = reconstruct_one_point(p1n, p2n, P1, P2)
                xy /= xy[3]
                cneu.append(xy[0:3])

            if len(cneu) == 0:
                continue
            data = pd.DataFrame(cneu, columns=["x", "y", "z"])
            # print('before',data[["x", "y", "z"]].std())
            zscore = np.abs(stats.zscore(data[["x", "y", "z"]]))
            # print(zscore)
            zscore.fillna(0, inplace=True)
            # Identify outliers as students with a z-score greater than 3
            threshold = 3
            data = data[zscore <= threshold].dropna()
            # print('after',data[["x", "y", "z"]].std())
            if data['x'].std() > 2 or data['y'].std() > 2 or data['z'].std() > 2:
                # print(data)
                continue
            # print(data[(np.abs(stats.zscore(data)) < 2).all(axis=1)])
            median = data.mean().values
            new.append([bester, i[1], median[0], median[1], median[2]])
        # print(new)
        if len(new) == 4:
            new = pd.DataFrame(new, columns=["marker", "ecke", "x", "y", "z"]).set_index(
                ["marker", "ecke"])
            coords = pd.concat([coords, new])
            return True, coords
    return False, coords


neue_koordinaten(coords, pictures, kameras)

In [ ]:
# pictures.loc[10, ['t1', 't2', 't3', 'r1', 'r2', 'r3']] = [None, None, None,None,None,None]

In [ ]:
treffer, coords = neue_koordinaten(coords, pictures, kameras)
treffer

In [ ]:
coords

In [ ]:
pictures = neues_bild(coords, pictures, kameras, corners)

In [ ]:
print(len(coords))
treffer, coords = neue_koordinaten(coords, pictures, kameras)
print(len(coords))

In [ ]:

coords.dropna(inplace=True)
coords.drop_duplicates(inplace=True)
# coords.drop(1, inplace=True)
coords.groupby("marker").count()

In [ ]:
wolke_zeigen(coords, pictures)

### Ausgleichung

In [ ]:
def passpunkt_groesse(coords):
    l = []
    l_beschriftung = []
    A = []
    m = 3.4
    for marker in coords.reset_index()['marker'].unique():
        # print(marker)
        if marker >= 24 and marker <= 25:
            m = 7.2
        elif marker >= 26:
            m = 4.0
        else:
            m = 3.4
        m1 = np.where(coords.index == (marker, 0))[0][0]
        m2 = np.where(coords.index == (marker, 1))[0][0]
        m3 = np.where(coords.index == (marker, 2))[0][0]
        m4 = np.where(coords.index == (marker, 3))[0][0]
        l_beschriftung.append("m" + str(marker) + "e1-2")
        l_beschriftung.append("m" + str(marker) + "e1-3")
        l_beschriftung.append("m" + str(marker) + "e1-4")
        l_beschriftung.append("m" + str(marker) + "e2-3")
        l_beschriftung.append("m" + str(marker) + "e2-4")
        l_beschriftung.append("m" + str(marker) + "e3-4")
        l.append(m)
        l.append(m*2**0.5)
        l.append(m)
        l.append(m)
        l.append(m*2**0.5)
        l.append(m)
        A.append([m1*2, m1*2 + 1, m2*2, m2*2+1])
        A.append([m1*2, m1*2 + 1, m3*2, m3*2+1])
        A.append([m1*2, m1*2 + 1, m4*2, m4*2+1])
        A.append([m2*2, m2*2 + 1, m3*2, m3*2+1])
        A.append([m2*2, m2*2 + 1, m4*2, m4*2+1])
        A.append([m3*2, m3*2 + 1, m4*2, m4*2+1])
    l = np.array(l)
    # print(l)
    A = np.array(A)
    return A, l, l_beschriftung

In [ ]:
def bekannte_punkte(coords):
    l = []
    l_beschriftung = []
    A = []
    for id, i in coords_wuerfel.reset_index().iterrows():
        l_beschriftung.append(str(i['marker']) + '-' + str(i['ecke'])+'x')
        l_beschriftung.append(str(i['marker']) + '-' + str(i['ecke'])+'y')
        l_beschriftung.append(str(i['marker']) + '-' + str(i['ecke'])+'z')
        l.append(i['x'])
        l.append(i['y'])
        l.append(i['z'])
        m1 = np.where(coords.index == (i['marker'], i['ecke']))[0][0]
        A.append([m1*3])
        A.append([m1*3 + 1])
        A.append([m1*3 + 2])
    l = np.array(l)
    A = np.array(A)
    return A, l.ravel(), l_beschriftung

In [ ]:
def ausgleichung(coords, pictures, kameras, corners):
    lp = corners.join(coords, lsuffix="_img", rsuffix="").join(
        pictures, lsuffix="", rsuffix="_cam").dropna()

    Apg, lpg, lpg_beschriftung = passpunkt_groesse(coords)
    Abp, lbp, lbp_beschriftung = bekannte_punkte(coords)

    kamera = kameras.reset_index().values
    kamera_ids = np.array(kamera[:, 0], dtype=np.int32)

    bilder = pictures.reset_index().dropna().drop(
        'pfad', axis=1).to_numpy(dtype=np.float32)
    bilder_ids = np.array(bilder[:, :2], dtype=np.int32)

    passpunkte = coords.reset_index().values
    passpunkte_ids = np.array(passpunkte[:, :2], dtype=np.int32)

    messung = lp.reset_index().to_numpy()[:, :5]
    messung_ids = np.array(messung[:, :3], dtype=np.int32)

    l = np.hstack(
        (messung[:, -2:].astype(np.float32).ravel(), lpg.ravel(), lbp.ravel()), dtype=np.float32)
    l_beschriftung = np.hstack(
        ([str(i[0])+'-'+str(i[1])+'-'+str(i[2]) for i in messung_ids], lpg_beschriftung, lbp_beschriftung))

    x0 = np.hstack((kamera[:, 1:].ravel(),
                    bilder[:, 2:].ravel(),
                    passpunkte[:, 2:].ravel()), dtype=np.float32)

    A = lil_matrix((len(l), len(x0)), dtype=int)

    n_camera = len(kamera)
    n_bilder = len(bilder)
    n_passpunkte = len(passpunkte)
    n_messungen = len(messung)
    messung_bild_id = np.empty(n_messungen, dtype=np.int32)
    messung_kamera_id = np.empty(n_messungen, dtype=np.int32)
    messung_passpunkt_id = np.empty(n_messungen, dtype=np.int32)
    num_cam_param = 9
    num_bild_param = 6
    num_pass_param = 3

    offset = n_camera*num_cam_param + n_bilder * num_bild_param

    for i, a in enumerate(Apg):
        zeile = n_messungen + i
        for j in a:
            A[zeile, offset+j] = 1

    for i, a in enumerate(Apg):
        zeile = n_messungen + len(lpg) + i
        for j in a:
            A[zeile, offset+j] = 1

    for i, m in enumerate(messung):
        # print(m)
        bild_id, = np.where(bilder_ids[:, 0] == m[2])
        bild_id = bild_id[0]
        messung_bild_id[i] = bild_id

        camera_id_array,  = np.where(kamera_ids[:] == bilder[bild_id, 1])
        camera_id = camera_id_array[0]
        messung_kamera_id[i] = camera_id

        passpunkt_id_array, = np.where(
            (passpunkte_ids[:, 0] == m[0]) & (passpunkte_ids[:, 1] == m[1]))
        passpunkt_id = passpunkt_id_array[0]
        messung_passpunkt_id[i] = passpunkt_id

        offset = camera_id*num_cam_param
        A[2*i:2*i+2, offset:offset+num_cam_param] = 1

        offset = n_camera * num_cam_param + bild_id * num_bild_param
        A[2*i:2*i+2, offset:offset + num_bild_param] = 1

        offset = n_camera*num_cam_param + n_bilder * \
            num_bild_param + passpunkt_id * num_pass_param
        A[2*i:2*i+2, offset:offset + num_pass_param] = 1

    def project(x0: NDArray[np.float32]) -> NDArray[np.float32]:
        p = np.empty(len(l), dtype=np.float32)

        K = []
        dist = []
        for i in range(n_camera):
            offset = i * num_cam_param
            fx = x0[offset]
            fy = x0[offset + 1]
            cx = x0[offset + 2]
            cy = x0[offset + 3]
            K.append(np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]]))
            dist.append(np.array([x0[offset + 4], x0[offset + 5],
                                  x0[offset + 6], x0[offset + 7], x0[offset + 8]]))

        r = []
        t = []
        for i in range(n_bilder):
            offset = n_camera * num_cam_param + \
                i*num_bild_param

            t.append(x0[offset:offset+3])
            r.append(x0[offset+3:offset+6])

        X = []
        for i in range(n_passpunkte):
            offset = n_camera * num_cam_param + \
                n_bilder*num_bild_param + \
                num_pass_param * i
            X.append(x0[offset:offset+3].reshape(1, 3))

        for i, m in enumerate(messung):
            punkt, _ = cv2.projectPoints(
                X[messung_passpunkt_id[i]], r[messung_bild_id[i]], t[messung_bild_id[i]], K[messung_kamera_id[i]], dist[messung_kamera_id[i]])
            # print(i*2+1)
            p[i*2] = punkt[0, 0, 0]
            p[i*2+1] = punkt[0, 0, 1]

        offset = n_messungen * 2
        for i, marker in enumerate(coords.reset_index()['marker'].unique()):
            # print(marker)
            if marker == 0:
                m = 15.6
            elif marker >= 24:
                m = 7.2
            else:
                m = 3.4
            m1 = np.where(coords.index == (marker, 0))[0][0]
            m2 = np.where(coords.index == (marker, 1))[0][0]
            m3 = np.where(coords.index == (marker, 2))[0][0]
            m4 = np.where(coords.index == (marker, 3))[0][0]

            p[offset+i +
                0] = np.linalg.norm(coords.values[m1] - coords.values[m2])
            p[offset+i +
                1] = np.linalg.norm(coords.values[m1] - coords.values[m3])
            p[offset+i +
                2] = np.linalg.norm(coords.values[m1] - coords.values[m4])
            p[offset+i +
                3] = np.linalg.norm(coords.values[m2] - coords.values[m3])
            p[offset+i +
                4] = np.linalg.norm(coords.values[m2] - coords.values[m4])
            p[offset+i +
                5] = np.linalg.norm(coords.values[m3] - coords.values[m4])
            # print(p[i:i+6])

        offset = n_messungen * 2 + len(lpg)
        for id, i in coords_wuerfel.reset_index().iterrows():
            pos = np.where(coords.index == (i['marker'], i['ecke']))[0][0]
            p[offset + 3*id] = x0[pos*3]
            p[offset + 3*id+1] = x0[pos*3+1]
            p[offset + 3*id+2] = x0[pos*3+2]
        # print(p)
        # print(np.array(p.ravel()-l, dtype=np.float32))
        return np.array(l-p.ravel(), dtype=np.float32)
    # print(x0)
    # print(l)
    print(project(x0).reshape(-1, 2))
    res = least_squares(project, x0, jac_sparsity=A, verbose=2,
                        x_scale='jac', method='trf', xtol=1e-10)  # type: ignore

    x = res.x

    kameras = pd.DataFrame(np.c_[kamera_ids, x[:num_cam_param * n_camera].reshape(n_camera, num_cam_param)],
                           columns=["kamera", "fx", "fy", "cx", "cy", "d1", "d2", "d3", "d4", "d5"]).astype({"kamera": int}).set_index("kamera")

    offset = num_cam_param * n_camera
    # print(bilder_ids[:, 1])
    neue_pictures = pd.DataFrame(np.c_[bilder_ids[:, 0], x[offset:offset+num_bild_param*n_bilder].reshape(
        n_bilder, num_bild_param)], columns=["img", "x", "y", "z", "r1", "r2", "r3"]).astype({"img": int}).set_index("img")
    # print(neue_pictures)
    pictures.update(neue_pictures)

    offset = num_cam_param * n_camera + num_bild_param*n_bilder
    coords = pd.DataFrame(np.c_[passpunkte_ids, x[offset:offset+num_pass_param*n_passpunkte].reshape(n_passpunkte, num_pass_param)],
                          columns=["marker", "ecke", "x", "y", "z"]).astype({"marker": int, "ecke": int}).set_index(["marker", "ecke"])

    # print(project(x).reshape(-1, 2))
    return res, kameras, pictures, coords, l, x0, project(x0), l_beschriftung

In [ ]:
vorher_coords = coords.copy()
vorher_pictures = pictures.copy()
vorher_kameras = kameras.copy()

In [ ]:
coords = vorher_coords.copy()
pictures = vorher_pictures.copy()
kameras = vorher_kameras.copy()

In [ ]:
# kameras, pictures, coords
# res, kameras, pictures, coords, l, x0, px0, l_beschriftung =
ausgleichung(
    coords, pictures, kameras, corners)

#### Kontrolle

In [ ]:
wolke_zeigen(coords, pictures)

In [ ]:
vorher_kameras

In [ ]:
kameras

In [ ]:
# coords.drop(25, inplace=True)

In [ ]:
list(zip(l_beschriftung, px0.astype(np.int32)))

In [ ]:
res

In [ ]:
pictures.loc[8, ["t1", "t2", "t3", "r1", "r2", "r3"]] = [
    None, None, None, None, None, None]

In [ ]:
def zeige_auf_bildern(coords, pictures, kameras):
    picWC = pictures.dropna().copy()
    picWC[['x', 'y', 'z']] = [-(cv2.Rodrigues(np.array(list(p[['r1', 'r2', 'r3']].values)))[
                                0]).T @ p[['t1', 't2', 't3']].values.T for i, p in pictures.dropna().iterrows()]

    for nr in picWC.index:
        print(nr)
        imgCV = cv2.imread(pictures.query("img == "+str(nr))['pfad'].values[0])
        r = pictures.query(
            "img == "+str(nr))[["r1", "r2", "r3"]].to_numpy(dtype=np.float64)[0]
        t = pictures.query(
            "img == "+str(nr))[["t1", "t2", "t3"]].to_numpy(dtype=np.float64)
        kamera = pictures.query("img == "+str(nr))['kamera'].values[0]
        print(kamera)
        kp = kameras.query("kamera == " + str(kamera)).values
        mtx = np.array([[kp[0][0], 0, kp[0][2]], [
            0, kp[0][1], kp[0][3]], [0, 0, 1]])
        dist = kp[0][4:]
        for p in corners.query("img == "+str(nr)).reset_index().values:
            cv2.circle(imgCV, (int(p[3]), int(p[4])), 10, (255, 0, 0), -1)
            cv2.putText(imgCV, str(int(p[1]))+'-'+str(int(p[2])), (int(p[3]),
                        int(p[4])),  cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0, 0), 5)
        img_points, _ = cv2.projectPoints(coords.values, r, t, mtx, dist)
        for c, p in zip(coords.reset_index()['marker'].values, img_points):
            try:
                cv2.circle(imgCV, (int(p[0][0]), int(
                    p[0][1])), 10, (0, 0, 255), -1)
                cv2.putText(imgCV, str(c), (int(p[0][0]), int(
                    p[0][1])), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 2)
            except:
                pass

        cam_points, _ = cv2.projectPoints(
            picWC[['x', 'y', 'z']].values, r, t, mtx, dist)
        for c, p in zip(picWC.reset_index()['img'].values, cam_points):
            if nr == c:
                continue
            try:
                cv2.circle(imgCV, (int(p[0][0]), int(
                    p[0][1])), 10, (0, 255, 0), -1)
                cv2.putText(imgCV, str(c+1), (int(p[0][0]), int(
                    p[0][1])), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 0), 2)
            except:
                pass
        cv2.drawFrameAxes(imgCV, mtx, dist, r, t, 10, 10)
        imgCV = cv2.undistort(imgCV, mtx, dist)
        plt.figure(figsize=(14, 11))
        plt.imshow(imgCV)
        plt.show()


zeige_auf_bildern(coords, pictures, kameras)

In [ ]:
U, s, Vh = np.linalg.svd(res.jac.todense(), full_matrices=False)
tol = np.finfo(float).eps*s[0]*max(res.jac.shape)
w = s > tol
cov = (Vh[w].T/s[w]**2) @ Vh[w]  # robust covariance matrix
perr = np.sqrt(np.diag(cov))
perr

In [ ]:
# coords.to_csv("wuerfel_coords.csv")
# pictures.to_csv("wuerfel_pictures.csv")
# kameras.to_csv("wuerfel_kameras.csv")

In [ ]:
kameras

In [ ]:
3386.344186*0.014

In [ ]:
kameras['fx'].values[0]*0.014

In [ ]:
1/(1/4.74-1/400)

In [ ]:
1/4000/0.014

In [ ]:
(vorher_coords-coords).std()

In [ ]:
pictures

In [ ]:
kameras

In [ ]:
coords.to_csv("wuerfel_coords_neu.csv")

In [ ]:
pictures.to_csv("wuerfel_pictures_neu.csv")